In [1]:
import pandas as pd
import sqlite3
# Set pandas to display all columns
pd.set_option('display.max_columns', None)

In [2]:
# Connect to an in-memory database
conn = sqlite3.connect(':memory:')

# If database is not created, create it with the following schema:
# - table 'summary' with columns 'survivedCount' as int
conn.execute('''
CREATE TABLE summary (
    survivedCount int
);
''')
print('Table summary created')

Table summary created


In [3]:
a = pd.read_csv('data.csv')

In [4]:
b = pd.read_csv('titanic.csv')

In [5]:
b.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
# Get the count of survived passengers in titanic
survived = int(b['Survived'].sum())
print(survived)

342


In [15]:
# Increment the count of survived passengers in the database
conn.execute('''
INSERT INTO summary (survivedCount) VALUES (?);
''', (survived, ))


In [16]:

# Check the count of survived passengers in the database
values = conn.execute('''
SELECT * FROM summary;
''').fetchall()

print(values)

[(342,)]


In [17]:
# See the table content
print(pd.read_sql_query('''
SELECT * FROM summary;
''', conn))

   survivedCount
0            342


In [ ]:

import pandas as pd
import sqlite3

def connect_to_database():
    conn = sqlite3.connect(':memory:')
    conn.execute('''
    CREATE TABLE summary (
        survivedCount int
    );
    ''')
    return conn

def read_titanic_data(file_path):
    b = pd.read_csv(file_path)
    survived = int(b['Survived'].sum())
    return survived

def update_and_query_database(conn, survived):
    conn.execute('''
    INSERT INTO summary (survivedCount) VALUES (?);
    ''', (survived, ))
    values = conn.execute('''
    SELECT * FROM summary;
    ''').fetchall()
    return values

def test_update_and_query_database():
    conn = connect_to_database()
    survived = read_titanic_data('titanic.csv')
    expected_values = update_and_query_database(conn, survived)
    conn.close()
    # Assuming the test database is in the same state as the production database
    conn_test = connect_to_database()
    survived_test = read_titanic_data('titanic.csv')
    result_values = update_and_query_database(conn_test, survived_test)
    conn_test.close()
    assert expected_values == result_values

conn = connect_to_database()
survived = read_titanic_data('titanic.csv')
values = update_and_query_database(conn, survived)
print(values)
test_update_and_query_database()
conn.close()

